## Libraries Importing

In [1]:
import warnings
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

In [2]:
import os
import shutil #recursively moves a file from src to dest and vice versa
import cv2 as cv #resize image , GF
import numpy as np
from tensorflow.keras import utils

In [3]:
def create_dir(in_dir, out_dir, gdir_1):
    for label in os.listdir(path= in_dir):
        if not os.path.exists(os.path.join(out_dir, gdir_1, label)):
            os.makedirs(os.path.join(out_dir, gdir_1, label))

## Gabor filter applying process

In [4]:
def preprocess_image(in_dir = 'input', out_dir= 'processed'):
    gdir_1 = 'Gabor_Images'
    
    create_dir(in_dir, out_dir, gdir_1)
    
    for label in os.listdir(path= in_dir):
        print('Processing started for : ',label)
        for image_name in os.listdir(os.path.join(in_dir,label)):
            if not image_name.endswith('.db'):
                in_path = os.path.join(in_dir, label, image_name)
                out_path_1 = os.path.join(out_dir, gdir_1, label, image_name) #path of gabor1
                img = cv.imread(in_path,0) #reading image 
                gabor_1 = cv.getGaborKernel((18, 18), 1.5, np.pi/4, 5.0, 1.5, 0, ktype=cv.CV_32F) #initialising the parameters of gabor filter 
                filtered_img_1 = cv.filter2D(img, cv.CV_8UC3, gabor_1) # applying gabor filter
                cv.imwrite(out_path_1, filtered_img_1) #writing filtered image in out_oath
        print('------ Processing done ------')

In [5]:
preprocess_image()

Processing started for :  no
------ Processing done ------
Processing started for :  yes
------ Processing done ------


In [6]:
input_dir = os.path.join('processed', 'Gabor_Images')
print('input_dir :', input_dir)

input_dir : processed/Gabor_Images


In [7]:
no_samples = 0
labels = os.listdir(input_dir)
print('Class labels : ', labels)

for label in labels:
    total = len(os.listdir(os.path.join(input_dir, label)))
    print(label, total)
    no_samples +=total
print('-'*30)
print('Total no of samples: ',no_samples)
print('-'*30)

Class labels :  ['no', 'yes']
no 98
yes 155
------------------------------
Total no of samples:  253
------------------------------


In [8]:
img_rows = 256
img_cols = 256
channels = 3

In [9]:
def load_data(input_dir, no_samples, labels):
    
    images = np.ndarray((no_samples, img_rows, img_cols, channels), dtype = np.float32)
    targets = np.zeros((no_samples,), dtype = np.uint8)
    
    i = 0
    print('-'*30)
    print('Loading training images...')
    print('-'*30)
    
    for j, label in enumerate(labels):
        image_names = os.listdir(os.path.join(input_dir, label))
        total = len(image_names)
        print(label, total)
        for image_name in image_names:
            img = cv.imread(os.path.join(input_dir, label, image_name), 1)
            img = np.array(cv.resize(img, (img_rows, img_cols)))
            
            images[i] = np.reshape(a= img, newshape= (img_rows, img_cols, channels))
            targets[i] = j
            
            i += 1
    print('Loading done.')
    
    targets = utils.to_categorical(y= targets, num_classes= len(labels))
    
    return images, targets

In [10]:
X, Y = load_data(input_dir, no_samples, labels)

------------------------------
Loading training images...
------------------------------
no 98
yes 155
Loading done.


In [11]:
print(X.shape, Y.shape)

(253, 256, 256, 3) (253, 2)


In [12]:
np.savez(file= 'dataset/data', x= X, y= Y)